## Tiled object detection from a video file
This notebook is an example of how to use DeGirum PySDK to do tiled object detection of a video stream from a video file.
Each video frame is divided by tiles with some overlap, each tile of the AI model input size (to avoid resizing).
Object detection is performed for each tile, then results from different tiles are combined.

For comparison purpose, non-tiled object detection with the same model is performed on the same video.
Results of tiled and non-tiled object detection are then combined on a single video.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `inference_host_address` option.

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify video file name, model name, and other options here

In [ ]:
# inference_host_address: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# model_name: name of the model for running AI inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# classes: list of classes to show
# *_ann_path: paths to save annotated videos
inference_host_address = "@cloud"
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Traffic.mp4"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"
model_zoo_url = "degirum/public"
classes = {"car"}
tiled_ann_path = "temp/tiled_object_detection.mp4"
non_tiled_ann_path = "temp/non-tiled_object_detection.mp4"
combined_ann_path = "temp/combined_object_detection.mp4"

#### The rest of the cells below should run without any modifications

In [ ]:
import numpy as np, degirum as dg, degirum_tools

# load object detection model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=model_zoo_url,
    token=degirum_tools.get_token(),
    output_class_set=classes,
    overlay_show_labels=False,
    overlay_show_probabilities=False,
    overlay_line_width=1,
    overlay_color=(0, 255, 0),
)

with degirum_tools.open_video_stream(video_source) as video_stream:
    model_size = model.model_info.InputW + model.model_info.InputH
    frame_size = degirum_tools.get_video_stream_properties(video_stream)[:2]

    # calculate tiles for tiled inference
    tiles = degirum_tools.generate_tiles_fixed_size(
        model_size, frame_size, min_overlap_percent=5.0
    )
    tiles = tiles[0] # pick top row of tiles

    # define tile extractor pseudo-model
    tile_extractor = degirum_tools.RegionExtractionPseudoModel(tiles, model)

    # define NMS options; for tiling, the best approach is to use IoS 
    # instead of IoU and use LARGEST_AREA box selection policy
    nms_options = degirum_tools.NmsOptions(
        threshold=0.3,
        use_iou=False,
        box_select=degirum_tools.NmsBoxSelectionPolicy.LARGEST_AREA,
    )

    # define compound model, which combines tile extractor and object detection model
    compound_model = degirum_tools.CroppingAndDetectingCompoundModel(
        tile_extractor, model, nms_options=nms_options
    )

    # run tiled inference on video stream
    degirum_tools.annotate_video(compound_model, video_stream, tiled_ann_path)

In [ ]:
# run regular inference on video stream
model.overlay_color=(255, 0, 0)
degirum_tools.annotate_video(model, video_source, non_tiled_ann_path)

In [ ]:
# combine two annotated videos into single video for comparison
import cv2

with degirum_tools.open_video_stream(non_tiled_ann_path) as non_tiled_stream:
    with degirum_tools.open_video_stream(tiled_ann_path) as tiled_stream:
        with degirum_tools.open_video_writer(
            combined_ann_path,
            *degirum_tools.get_video_stream_properties(tiled_stream)[:2]
        ) as writer:

            progress = degirum_tools.Progress(tiled_stream.get(cv2.CAP_PROP_FRAME_COUNT))
            for tiled_frame, non_tiled_frame in zip(
                degirum_tools.video_source(tiled_stream),
                degirum_tools.video_source(non_tiled_stream),
            ):
                # insert top half of non_tiled_frame into bottom half of tiled_frame
                half_height = tiled_frame.shape[0] // 2
                tiled_frame[half_height:, :] = non_tiled_frame[:half_height, :]
                writer.write(tiled_frame)
                progress.step()

In [ ]:
degirum_tools.ipython_display(combined_ann_path, True)